In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from datetime import timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/grocery-sales-forecast/train.csv',parse_dates=['date'], index_col='date').drop(columns=['id'])
df_test = pd.read_csv('../input/grocery-sales-forecast/test.csv',parse_dates=['date'], index_col='date')

df_test.head(2)

In [ ]:
num_days = dict(enumerate(sorted(list(set(df_train.index)))))
num_prod = dict(enumerate(sorted(list(set(df_train.product_id)))))
days_num = {v: k for k, v in num_days.items()}
prod_num = {v: k for k, v in num_prod.items()}
arr = np.zeros((len(days_num), len(prod_num), 2))

In [ ]:
for row in df_train.iterrows():
    arr[days_num[row[0]], prod_num[row[1].product_id], row[1].warehouse_id] += row[1].quantity

In [ ]:
otv_df=pd.DataFrame(columns=['id'])
for tek_prod_id in tqdm(df_test['product_id'].unique()):
    for tek_store_id in [0,1]:
        
        tek_test_df=df_test[df_test['product_id']==tek_prod_id]
        tek_test_df=tek_test_df[tek_test_df['warehouse_id']==tek_store_id]
        
        mas=arr[:,prod_num[tek_prod_id],tek_store_id]

        model3 = Holt(np.array(mas).astype(float)).fit()
        #model3=ExponentialSmoothing(np.asarray(mas).astype(float) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit()
        forecast=np.round(model3.forecast(len(tek_test_df)))
        
        newt=tek_test_df.copy()[['id']]
        newt['prediction'] = forecast
        otv_df = pd.concat((otv_df,newt), axis = 0,sort=True)
        
otv_df.shape

In [ ]:
new_df=otv_df.copy()
new_df=new_df.sort_values(by=['id'])
new_df.rename(columns={'prediction':'quantity'},inplace=True)
new_df['quantity']=new_df['quantity'].astype(int)
new_df.to_csv("./y_predTest.csv",index=False)
new_df.head(5)